In [ ]:
import os, sys
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta
import psycopg2
from datetime import datetime
from openpyxl import Workbook

In [ ]:
dfDrivers = prodFetch("""select id driverId, zoneId driverZone, penaltyWallet/100 as cummulativePenalty from drivers where id like 'D%' and clientId in ('BS00') and vehicleType not in ('e-2w')""")

In [ ]:
dfTxns = prodFetch("""
SELECT 
    t.driverId,
    t.zoneId,
    COUNT(DISTINCT CASE 
        WHEN t.date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY) THEN t.date 
        ELSE NULL 
    END) AS active_days,
    MAX(t.date) AS last_active_date,
    CASE 
        WHEN SUM(CASE WHEN t.wrongBatteries != '[]' THEN 1 ELSE 0 END) > 0 
        THEN 1 ELSE 0 
    END AS crossSwapFlag
FROM transactions t
WHERE t.deletedAt IS NULL
  AND t.clientId IN ('BS00')
  AND t.vehicleType NOT IN ('e-2w')
  AND t.date = DATE(DATE_ADD(t.createdAt, INTERVAL 330 MINUTE))
GROUP BY t.driverId, t.zoneId
HAVING last_active_date >= '2024-09-01';
""")

In [ ]:
dfTxns['isChurned'] = np.where((dfTxns['active_days'] == 0), 1, 0)

In [ ]:
dfTxns

In [ ]:
dfTickets = adbFetch("""
select issue_id, 
    complainant_id, 
    date, 
    complainant_zone, 
    issue_name, 
    category_id, 
    category_name
FROM ticketModelNew
WHERE 
    month in ('2024-11', '2024-12', '2025-01')
    AND complainant_type = 'driver'
ORDER BY date ASC;
                     """)

dfTickets

In [ ]:
dfTxns['TotalTicketsCount'] = dfTxns['driverId'].map(dfTickets['complainant_id'].value_counts()).fillna(0)

In [ ]:
dfTxns['SwapHistoryTicketsCount'] = dfTxns['driverId'].map(
    dfTickets[dfTickets['category_id'] == 64]['complainant_id'].value_counts()
).fillna(0)

In [ ]:
dfTxns['BatteryTicketsCount'] = dfTxns['driverId'].map(
    dfTickets[dfTickets['category_id'] == 68]['complainant_id'].value_counts()
).fillna(0)

In [ ]:
dfTxns['OtherTicketsCount'] = dfTxns['TotalTicketsCount'] - dfTxns['SwapHistoryTicketsCount']

In [ ]:
dfTxns

In [ ]:
dfTxns['isChurned'] = pd.to_numeric(dfTxns['isChurned'], errors='coerce').fillna(0).astype(int)

dfFiltered = dfTxns[dfTxns['isChurned'] == 1]

dfFiltered = dfFiltered.drop(columns=['isChurned'])

In [ ]:
dfFiltered

In [ ]:
write("https://docs.google.com/spreadsheets/d/1T4xuenK0bry8yfq00cjUXBdiPHuKwKNV6N6j9YHDtbA/edit?gid=1041762128#gid=1041762128", "python-data", dfFiltered)

In [ ]:
dfFinal = dfTickets.merge(dfFiltered, left_on='complainant_id', right_on='driverId', how='right')

In [ ]:
dfFinal

In [ ]:
dfFinal = dfFinal.drop(columns=[
    'driverId', 'zoneId', 'active_days', 'last_active_date', 
    'crossSwapFlag', 'TotalTicketsCount', 
    'SwapHistoryTicketsCount', 'OtherTicketsCount', 'BatteryTicketsCount'
])

dfFinal

In [ ]:
dfFinal.drop_duplicates(inplace=True)

In [ ]:
dfFinal

In [ ]:
dfFinal = dfFinal.sort_values(by='date', ascending=True)

In [ ]:
dfFinal

In [ ]:
dfTxn = prodFetch("""
                  SELECT driverId, date
                  FROM transactions
                  WHERE deletedAt IS NULL
                    AND clientId IN ('BS00')
                    AND vehicleType NOT IN ('e-2w')
                    AND date >= '2024-09-01'
                    and date <= '2025-01-06'
                  """)

In [ ]:
dfTxn['date'] = pd.to_datetime(dfTxn['date'])

dfLastActive = dfTxn.groupby('driverId')['date'].max().reset_index()
dfLastActive.rename(columns={'date': 'last_active_date'}, inplace=True)

dfTxn = dfTxn.merge(dfLastActive, on='driverId', how='left')

dfTxn['30_days_before_last_active'] = dfTxn['last_active_date'] - pd.Timedelta(days=30)

dfTxn['ops_days_30'] = dfTxn.apply(
    lambda row: dfTxn[(dfTxn['driverId'] == row['driverId']) & 
                      (dfTxn['date'] > row['30_days_before_last_active']) & 
                      (dfTxn['date'] <= row['last_active_date'])]['date'].nunique(),
    axis=1
)

dfTxns = dfTxn.merge(dfDrivers[['driverId', 'cummulativePenalty']], on='driverId', how='left')

dfFinal = dfFiltered.merge(dfTxns[['driverId', 'ops_days_30', 'cummulativePenalty']], left_on='driverId', right_on='driverId', how='left')

dfFinal[['driverId', 'ops_days_30', 'cummulativePenalty']]

In [ ]:
dfFinal = dfFinal.drop_duplicates(subset='driverId')
dfFinal

In [ ]:
write("https://docs.google.com/spreadsheets/d/1T4xuenK0bry8yfq00cjUXBdiPHuKwKNV6N6j9YHDtbA/edit?gid=1041762128#gid=1041762128", "final-data", dfFinal)